In [1]:
import numpy as np
import torch

from torch import nn

PATH = "/home/sagarpatel/Desktop/ece496-capstone/"

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
from video import Video

video_path = PATH + "train/sample/video.mp4"
transcript_path = PATH + "train/sample/transcript.vtt"
transcript2_path = PATH + "train/sample/transcript2.vtt"

v = Video(video_path, transcript_path)
v.align()
v.generate_frames("sample", swap=True)

v2 = Video(video_path, transcript2_path)
v2.align()
v2.generate_frames("sample", swap=True)

In [4]:
import glob
import itertools

from loss import loss_RA_MIL
from detector import Detector
from parser import parse

detector = Detector(device)

loading configuration file cache
loading weights file https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin from cache at /home/sagarpatel/.cache/torch/transformers/57f6df6abe353be2773f2700159c65615babf39ab5b48114d2b49267672ae10f.77b59256a4cf8343ae0f923246a81489fc8d82f98d082edc2d2037c977c0d9d0
All model checkpoint weights were used when initializing GeneralizedRCNN.

All the weights of GeneralizedRCNN were initialized from the model checkpoint at unc-nlp/frcnn-vg-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GeneralizedRCNN for predictions without further training.


In [5]:
import importlib
import loss
importlib.reload(loss)

<module 'loss' from '/home/sagarpatel/Desktop/ece496-capstone/train/loss.py'>

In [6]:
images = sorted(glob.glob(PATH + "train/sample/*.png"))
candidates = [detector.inference(image, max_detections=5) for image in images]

In [7]:
candidates[0][0].device

device(type='cpu')

In [ ]:
NULL = '[unused1]'
PAD = '[unused2]'
ENTITY = '[unused3]'
ACTION = '[SEP]'

MAX_STEP_LENGTH = 30

In [ ]:
steps1 = [step.text.strip() for step in v.steps]
steps2 = [step.text.strip() for step in v2.steps]

In [ ]:
steps1_flat = ' [SEP] '.join(steps1) + ' ' + NULL + ' [SEP]'
steps2_flat = ' [SEP] '.join(steps2) + ' ' + NULL + ' [SEP]'

In [ ]:
# Find all of the bounding boxes for the detections and their features.
boxes = torch.tensor([candidate[0].numpy() for candidate in candidates]).squeeze(1)
features = torch.tensor([candidate[1].numpy() for candidate in candidates]).squeeze(1)

boxes = boxes.flatten(start_dim=0, end_dim=1)
features = features.flatten(start_dim=0, end_dim=1)

boxes = torch.stack((boxes, boxes))
features = torch.stack((features, features))

In [ ]:
# https://github.com/google-research/bert/issues/635
# https://colab.research.google.com/drive/18TyuMfZYlgQ_nXo-tr8LCnzUaoX0KS-h?usp=sharing#scrollTo=W4cZIVrg82ua.

import itertools
import torch
import einops

from transformers import LxmertModel, LxmertTokenizer
from torch import nn
from torch.nn.utils.rnn import pad_sequence

class Model(nn.Module):
    NULL = '[unused1]'
    PAD = '[unused2]'
    ENTITY = '[unused3]'
    ACTION = '[SEP]'
    
    DETECTION_EMBEDDING_SIZE = 2048
    OUTPUT_EMBEDDING_SIZE = 768
        
    def __init__(self, device):
        super(Model, self).__init__()
        
        self.lxmert_tokenizer = LxmertTokenizer.from_pretrained("unc-nlp/lxmert-base-uncased")
        self.lxmert_tokenizer.add_special_tokens({"additional_special_tokens": [NULL, PAD, ENTITY]})
        self.lxmert_tokenizer.encode([NULL, PAD, ENTITY])
        
        self.NULL_TOKEN = self.lxmert_tokenizer.convert_tokens_to_ids(NULL)
        self.ENTITY_TOKEN = self.lxmert_tokenizer.convert_tokens_to_ids(ENTITY)
        self.ACTION_TOKEN = self.lxmert_tokenizer.convert_tokens_to_ids(ACTION)
        
        self.lxmert = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased")
        self.lxmert.to(device)
        
    def forward(self, BATCH_SIZE, NUM_ACTIONS, CANDIDATES, steps, features, boxes, entity_count):
        '''
            MAX_INSTRUCTION_LENGTH
                : maximum number of words in a combined string of steps for a video,
                  this instruction string must contain the NULL step.
                  
                  ex. | E ... E . [SEP] [PAD] [PAD] | .... [SEP] [PAD] | NULL | 
                  
            steps
                : batched video steps of size (BATCH_SIZE, NUM_ACTIONS), list of lists,
                  each instructional video must have the same number of steps = NUM_ACTIONS.
                
                  ex. [['step 1.1', 'step 1.2'], ['step 2.1', 'step 2.2']]
                
            features
                : batched detection features of size (BATCH_SIZE, CANDIDATES * NUM_ACTIONS, 2048)
                
            boxes
                : batched bounding boxes of detection features of size (BATCH_SIZE, CANDIDATES * NUM_ACTIONS, 4)
                
            entity_count:
                : number of entities per action, size (BATCH_SIZE, NUM_ACTIONS), list of lists
        '''

        inputs = self.lxmert_tokenizer(
            steps,
            padding="longest",
            truncation=False,
            return_token_type_ids=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

        output = self.lxmert(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            visual_feats=features,
            visual_pos=boxes,
            token_type_ids=inputs.token_type_ids,
            return_dict=True,
            output_attentions=True
        )
        
        token_ids = inputs.input_ids
        
        entity_idx = ((token_ids == self.ENTITY_TOKEN) | (token_ids == self.NULL_TOKEN))
        action_idx = (token_ids == self.ACTION_TOKEN)
        
        entity_embeddings = output['language_output'][entity_idx]
        action_embeddings = output['language_output'][action_idx]
        vision_embeddings = output['vision_output']
        
        split_sizes = torch.tensor(entity_count).flatten().tolist()
        entities = entity_embeddings.split(split_sizes)
        
        E = pad_sequence(entities, batch_first=True)
        max_entities = E.shape[1]
        E = E.reshape(-1, NUM_ACTIONS, E.shape[1], E.shape[2])

        A = action_embeddings.reshape(BATCH_SIZE, NUM_ACTIONS, -1)
        V = vision_embeddings.reshape(BATCH_SIZE, NUM_ACTIONS, CANDIDATES, -1)
        
        # Calculate loss_E.
        loss_E = E
        
        # Calculate VG (VG_scores_index) and loss_V.
        VG_scores = torch.einsum('bacs, baes -> baec', V, E)
        VG_scores_max, VG_scores_index = VG_scores.max(dim=-1)

        V_flat = V.reshape(-1, self.OUTPUT_EMBEDDING_SIZE)

        VG_scores_index_flat = VG_scores_index.flatten()
        offsets = torch.arange(0, BATCH_SIZE * NUM_ACTIONS * CANDIDATES, CANDIDATES)
        offsets = offsets.repeat_interleave(max_entities)

        VG_scores_index_flat = VG_scores_index_flat + offsets

        loss_V = V_flat[VG_scores_index_flat, :].reshape(BATCH_SIZE, NUM_ACTIONS, max_entities, -1)
        
        # Calculate RR (RR_scores_index).
        #note: we write c here as number of actions as well
        #since we don't wish to sum over a 
        RR_scores = torch.einsum('baes, bcs -> baec', E, A)

        edge_mask = torch.ones(NUM_ACTIONS, NUM_ACTIONS).tril(diagonal=-1)
        edge_mask[-1, :] = 0
        edge_mask[:, -1] = 1
        edge_mask = einops.repeat(edge_mask, 'x y -> b x c y', b=BATCH_SIZE, c=max_entities)

        RR_scores_max, RR_scores_index = (RR_scores * edge_mask).max(dim=-1)
        
        # Calculate loss_R.
        loss_R = torch.ones((BATCH_SIZE, NUM_ACTIONS, NUM_ACTIONS))

        dim_1 = RR_scores_index.reshape(BATCH_SIZE, -1)
        dim_2 = torch.arange(NUM_ACTIONS).repeat_interleave(max_entities)
        dim_2 = einops.repeat(dim_2, 'd -> b d', b=BATCH_SIZE)

        loss_R[:, dim_1, dim_2] = 0.0
        
        #entity embeddings, selected visual grounding embeddings, adjacency list for
        #ref resolution
        return loss_E, loss_V, loss_R, VG_scores_index, RR_scores_index, A, E, V

In [ ]:
steps = [steps1_flat, steps2_flat]

ENTITIES_COUNT = [
    [2, 2, 1, 2, 3, 2, 2, 2, 1, 1],
    [1, 1, 1, 1, 3, 2, 1, 2, 1, 1]
]

In [ ]:
model = Model(device)
model.to(device)
model.eval()

features.to(device)
boxes.to(device)

In [ ]:
loss_E, loss_V, loss_R, VG, RR, A, E, V = model(2, 10, 5, steps, features, boxes, ENTITIES_COUNT)

# https://stackoverflow.com/questions/14531346/how-to-add-a-text-into-a-rectangle

# Visualization. Requires: 
# - list of entities and entity count
# - list of steps
# - VG, RR, bb -- Note: this is for the current video, ie. not batched
# - directory with an image per step (0.png, 1.png, etc.)

bb = boxes[0].reshape([10, 5, 4]) # Actions, candidates, bb

VG = VG[0]
RR = RR[0]

entities = [
    ["the tomatoes", "a pan"],
    ["oil", "the pan"],
    ["the bacon"],
    ["some mayonnaise", "the bread"],
    ["a piece of", "lettuce", "it"],
    ["the tomatoes", "it"],
    ["some salt", "it"],
    ["the bacon", "the top"],
    ["piece of bread"],
    ["NULL"]
]

entity_count = ENTITIES_COUNT[0]

steps_trimmed = [step.replace(ENTITY, "").replace("  ", " ") for step in steps1]
steps_trimmed.append("NULL")

directory = "/home/sagarpatel/Desktop/ece496-capstone/train/sample/"

import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from IPython.display import display

def visualize(VG, RR, bb, entities, steps_trimmed, steps, directory):
    # VG visualizer.
    for step_idx, data in enumerate(zip(entity_count, VG, bb)):
        step_entity_count, step_VG, step_bb = data
        frame = cv2.imread('/home/sagarpatel/Desktop/ece496-capstone/train/sample/{}.png'.format(step_idx))
        height = frame.shape[0]
        width = frame.shape[1]

        fig = plt.figure()
        plt.imshow(frame)
        axes = plt.gca()

        print("\nStep {}\n---------".format(step_idx))

        for entity_idx, entity_VG in enumerate(step_VG):        
            if entity_idx >= step_entity_count:
                break

            entity_text = entities[step_idx][entity_idx]
            action_id = RR[step_idx][entity_idx]
            action_text = steps_trimmed[action_id]
            print("Entity: {} --> Action: {} ({})".format(entity_text, action_id, action_text))

            index = int(entity_VG)
            entity_bb = step_bb[index].tolist()

            x0, y0 = entity_bb[0] * width, entity_bb[1] * height
            x1, y1 = entity_bb[2] * width, entity_bb[3] * height
            
            width = x1 - x0
            height = y1 - y0

            box = Rectangle((x0, y0), width, height, linewidth=1, edgecolor='r', facecolor='none')
            axes.add_patch(box)
            axes.annotate(entity_text, (x0 + (width / 2), y0 + (height / 2)), color='white', 
                fontsize=10, ha='center', va='center')

In [ ]:
visualize(VG, RR, bb, entities, entity_count, steps, directory)

In [ ]:
def train(model, epochs=25, lr=0.001, batch_size=10, y=0.5):
    '''
    Training loop for the model.
    '''
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # Ouput losses.
    train_loss = np.zeros(epochs)
    
    # Output accuracies.
    train_accuracy = np.zeros(epochs)
    
    m_RR = None
    m_VG = None
    
    model.train()
    
    for epoch in range(epochs):
        # Zero out any gradients.
        optimizer.zero_grad()

        # Run inference (forward pass).
        loss_E, loss_V, loss_R = model(265, 2, 10, 5, steps, features, boxes, ENTITIES_COUNT)

        # Loss from alignment.
        loss_ = loss.loss_RA_MIL(y, loss_R, loss_E, loss_V)
        print(loss_)

        # Backpropagation (backward pass).
        loss_.backward()

        # Update parameters.
        optimizer.step()

        #m_RR = RR
        #m_VG = VG
            
        # TODO: save loss and accuracy at each epoch, plot (and checkpoint).
        train_loss[epoch] = loss_
        
    return train_loss

In [ ]:
model = Model()
train_loss = train(model, epochs=50, lr=0.0001)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_loss)

In [ ]:
model

In [ ]:
boxes.size()

In [ ]:
steps[0]

In [ ]:
model = Model()
loss_E, loss_V, loss_R = model(265, 2, 10, 5, steps, features, boxes, ENTITIES_COUNT)

In [ ]:
torch.tensor(loss_R)

In [ ]:
features.size()

In [ ]:
 MAX_INSTRUCTION_LENGTH, BATCH_SIZE, NUM_ACTIONS, CANDIDATES, steps, features, boxes, entity_count):
        '''
            MAX_INSTRUCTION_LENGTH
                : maximum number of words in a combined string of steps for a video,
                  this instruction string must contain the NULL step.
                  
                  ex. | E ... E . [SEP] [PAD] [PAD] | .... [SEP] [PAD] | NULL | 
                  
            steps
                : batched video steps of size (BATCH_SIZE, NUM_ACTIONS), list of lists,
                  each instructional video must have the same number of steps = NUM_ACTIONS.
                
                  ex. [['step 1.1', 'step 1.2'], ['step 2.1', 'step 2.2']]
                
            features
                : batched detection features of size (BATCH_SIZE, CANDIDATES * NUM_ACTIONS, 2048)
                
            boxes
                : batched bounding boxes of detection features of size (BATCH_SIZE, CANDIDATES * NUM_ACTIONS, 4)
                
            entity_count:
                : number of entities per action, size (BATCH_SIZE, NUM_ACTIONS), list of lists
        '''

In [ ]:
        inputs = tokenizer(
            steps,
            padding="max_length",
            max_length= 265 + 2, # [CLS] and [SEP] token
            truncation=False,
            return_token_type_ids=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

In [ ]:
len(steps[1].split(' '))

In [ ]:
inputs['input_ids']

In [ ]:
decoded_steps = tokenizer.batch_decode(inputs['input_ids'])

In [ ]:
len(decoded_steps[0].split(' '))

In [ ]:
lxmert = LxmertModel.from_pretrained("unc-nlp/lxmert-base-uncased")

In [ ]:
        output = lxmert(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            visual_feats=features,
            visual_pos=boxes,
            token_type_ids=inputs.token_type_ids,
            return_dict=True,
            output_attentions=True
        )

In [ ]:
output['language_output'].size()

In [ ]:
NUM_ACTIONS=10
BATCH_SIZE=2

edge_mask = torch.ones(NUM_ACTIONS, NUM_ACTIONS).tril(diagonal=-1)
edge_mask[-1, :] = 0
edge_mask[:, -1] = 1
edge_mask = einops.repeat(edge_mask, 'x y -> b x c y', b=BATCH_SIZE, c=5)

In [ ]:
edge_mask[0]

In [ ]:
a = torch.rand(2,10, 10)

In [ ]:
a.diagonal(dim1 = 1, dim2= 2).size()